In [28]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from tkinter import Tk, filedialog
import os
from glob import glob
from pathlib import Path

# Widgets for directory selection
directory_path = widgets.Text(description="Directory Path")
select_button = widgets.Button(description="Select Directory")
output = widgets.Output()

# Global variables
selected_directory = None
csv_files = []
video_files = []
h5_files = []

# Function to handle directory selection
def on_select_button_click(b):
    global selected_directory, csv_files, video_files, h5_files
    with output:
        clear_output(wait=True)
        root = Tk()
        root.attributes("-topmost", True)
        root.withdraw()  # Hide the main Tkinter window
        new_directory = filedialog.askdirectory(title="Please select a directory containing the .csv files")
        root.destroy()  # Destroy the Tkinter instance

        if new_directory:  # Check if a directory was selected
            selected_directory = new_directory
            directory_path.value = selected_directory
            print(f"You chose: {selected_directory}")
            
            video_files = glob(os.path.join(selected_directory, '**', '*.avi'), recursive=True)
            csv_files = glob(os.path.join(selected_directory, '**', '*.csv'), recursive=True)
            h5_files = glob(os.path.join(selected_directory, '**', '*.h5'), recursive=True)
            
            if csv_files:
                print(f"\nFound {len(csv_files)} .csv files in the directory and subdirectories")
                for csv_file in csv_files:
                    print(f"- {os.path.basename(csv_file)}")
            if video_files:
                print(f"\nFound {len(video_files)} .avi files in the directory and subdirectories")
                for video_file in video_files:
                    print(f"- {os.path.basename(video_file)}")
            if h5_files:
                print(f"\nFound {len(h5_files)} .h5 files in the directory and subdirectories")
                for h5_file in h5_files:
                    print(f"- {os.path.basename(h5_file)}")
            else:
                print("No files found in the selected directory.")
        else:
            print("No directory selected.")



# Connect button to function and display widgets
select_button.on_click(on_select_button_click)
display(directory_path, select_button, output)

Text(value='', description='Directory Path')

Button(description='Select Directory', style=ButtonStyle())

Output()

In [ ]:
# process csv and read in existing points 
from pathlib import Path

# Use the first CSV file and video file from your lists
csv_file = Path(csv_files[0])  # Use the first file in your csv_files list
video_file = Path(video_files[0])  # Use the first file in your video_files list

In [50]:
# Enable Qt event loop for Jupyter
%gui qt

from typing import List
from dask_image.imread import imread
from magicgui.widgets import ComboBox, Container
import napari
import numpy as np
import imageio


COLOR_CYCLE = [
    '#1f77b4',
    '#ff7f0e',
    '#2ca02c',
    '#d62728',
    '#9467bd',
    '#8c564b',
    '#e377c2',
    '#7f7f7f',
    '#bcbd22',
    '#17becf'
]

# Example usage
# Update this path to the image stack (e.g., "*.tif" or "*.png") you want to annotate
# im_path = im_path = Path(r'C:\Users\Windows10\Downloads\PointAnnotator-master\examples\m4s1\*.png')  # Replace with the actual image path
labels = ["FR", "FR_ti", "FR_ti1", "FR_tm", "FR_to1", "FR_to", 
 "FL", "FL_ti", "FL_ti1", "FL_tm", "FL_to1", "FL_to", 
 "HR", "HR_ti", "HR_ti1", "HR_tm", "HR_to1", "HR_to", 
 "HL", "HL_ti", "HL_ti1", "HL_tm", "HL_to1", "HL_to"]


# Function to extract points and labels from the DataFrame
def extract_points_and_labels(df):
    points = []
    labels_csv = []
    for frame_idx in range(len(df)):
        for bodypart in df.columns.levels[1]:  # Iterate over bodyparts
            try:
                x = df.loc[frame_idx, ('DLC_resnet50_allGeckosJan26shuffle1_800000', bodypart, 'x')]
                y = df.loc[frame_idx, ('DLC_resnet50_allGeckosJan26shuffle1_800000', bodypart, 'y')]
                points.append([frame_idx, y, x])  # Napari uses (frame, y, x) order
                labels_csv.append(bodypart)  # Add the bodypart as a label
            except KeyError:
                continue
    return np.array(points), labels

def create_label_menu(points_layer, labels):
    """Create a label menu widget that can be added to the napari viewer dock."""
    label_menu = ComboBox(label='feature_label', choices=labels)
    label_widget = Container(widgets=[label_menu])

    def update_label_menu(event):
        """Update the label menu when the point selection changes."""
        new_label = str(points_layer.current_properties['label'][0])
        if new_label != label_menu.value:
            label_menu.value = new_label

    points_layer.events.current_properties.connect(update_label_menu)

    def label_changed(selected_label):
        """Update the Points layer when the label menu selection changes."""
        current_properties = points_layer.current_properties
        current_properties['label'] = np.asarray([selected_label])
        points_layer.current_properties = current_properties
        points_layer.refresh_colors()

    label_menu.changed.connect(label_changed)
    return label_widget

# Function to load video frames as a NumPy array
def load_video_as_frames(video_path):
    reader = imageio.get_reader(video_path)  # Open the video file
    frames = [frame for frame in reader]  # Read all frames
    return np.stack(frames, axis=0)  # Convert list of frames to a NumPy array (T, H, W, C)

def point_annotator(
    im_path: str,
    labels: List[str],
):
    """Create a GUI for annotating points in a series of images.

    Parameters
    ----------
    im_path : str
        Glob-like string for the images to be labeled.
    labels : List[str]
        List of the labels for each keypoint to be annotated (e.g., the body parts to be labeled).
    """
    stack = load_video_as_frames(video_file)

    # Initialize the Napari viewer
    viewer = napari.view_image(stack)
    # Set the initial displayed frame to the first frame (index 0)
    viewer.dims.set_point(0, 0)  # Set the first dimension (frame index) to 0
    
    points_layer = viewer.add_points(
        ndim=3,
        property_choices={'label': labels},
        edge_color='label',
        edge_color_cycle=COLOR_CYCLE,
        symbol='o',
        face_color='transparent',
        edge_width=0.5,  # fraction of point size
        size=12,
    )
    points_layer.edge_color_mode = 'cycle'

    # Add the label menu widget to the viewer
    label_widget = create_label_menu(points_layer, labels)
    viewer.window.add_dock_widget(label_widget)

    @viewer.bind_key('.')
    def next_label(event=None):
        """Keybinding to advance to the next label with wraparound."""
        current_properties = points_layer.current_properties
        current_label = current_properties['label'][0]
        ind = list(labels).index(current_label)
        new_ind = (ind + 1) % len(labels)
        new_label = labels[new_ind]
        current_properties['label'] = np.array([new_label])
        points_layer.current_properties = current_properties
        points_layer.refresh_colors()

    def next_on_click(layer, event):
        """Mouse click binding to advance the label when a point is added."""
        if layer.mode == 'add':
            # By default, napari selects the point that was just added.
            # Disable that behavior, as the highlight gets in the way
            # and also causes next_label to change the color of the
            # point that was just added.
            layer.selected_data = set()
            next_label()

    points_layer.mode = 'add'
    points_layer.mouse_drag_callbacks.append(next_on_click)

    @viewer.bind_key(',')
    def prev_label(event):
        """Keybinding to decrement to the previous label with wraparound."""
        current_properties = points_layer.current_properties
        current_label = current_properties['label'][0]
        ind = list(labels).index(current_label)
        n_labels = len(labels)
        new_ind = ((ind - 1) + n_labels) % n_labels
        new_label = labels[new_ind]
        current_properties['label'] = np.array([new_label])
        points_layer.current_properties = current_properties
        points_layer.refresh_colors()
        
    @viewer.bind_key('Ctrl-d')
    def save_data(event=None):
        """Save data when Ctrl+d is pressed."""
        print("Ctrl+d pressed! Saving data...")
        # Add your save logic here
        


video_file = Path(video_files[0])  # Use the first file in your video_files list

point_annotator(video_file, labels)


C:\Users\Windows10\AppData\Local\Temp\ipykernel_14100\1335217013.py:101: FutureWarning: Argument 'edge_width' is deprecated, please use 'border_width' instead. The argument 'edge_width' was deprecated in 0.5.0 and it will be removed in 0.6.0.
  points_layer = viewer.add_points(
c:\ProgramData\anaconda3\envs\napari_standalone\Lib\site-packages\napari\utils\migrations.py:101: FutureWarning: Argument 'edge_color' is deprecated, please use 'border_color' instead. The argument 'edge_color' was deprecated in 0.5.0 and it will be removed in 0.6.0.
  return func(*args, **kwargs)
c:\ProgramData\anaconda3\envs\napari_standalone\Lib\site-packages\napari\utils\migrations.py:101: FutureWarning: Argument 'edge_color_cycle' is deprecated, please use 'border_color_cycle' instead. The argument 'edge_color_cycle' was deprecated in 0.5.0 and it will be removed in 0.6.0.
  return func(*args, **kwargs)
C:\Users\Windows10\AppData\Local\Temp\ipykernel_14100\1335217013.py:171: FutureWarning: Points.edge_color